In [1]:
import numpy as np
import cv2
import time
import os
import yaml

import open3d as o3d

import wire_detection_utils as wdu
import viz_utils as vu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('wire_detect_config.yaml', 'r') as file:
    detection_config = yaml.safe_load(file)

# Create a WireDetector instance
input_image_size = [480, 270]

# folder = "/media/tyler/Storage/field_tests/street_wire_experiments/wire_tracking_05-07_40fov/" 
folder = '/media/tyler/Storage/Research/Datasets/wire_tracking_05-07_40fov/'
# folder = "/home/tyler/Documents/wire_tracking_05-07_40fov/" 
# folder = "/media/tyler/Storage/field_tests/hawkins_2-19/wire_tracking_02-19_1/" 
# folder = "/media/tyler/Storage/field_tests/acfa_3-21/" 

rgb_folder = folder + 'rgb_images/'
depth_folder = folder + 'depth_images/'
camera_intrinsics_file = folder + 'rgb_camera_intrinsics.npy'
camera_intrinsics = np.load(camera_intrinsics_file)
rgb_file_list = sorted(os.listdir(rgb_folder))
depth_file_list = sorted(os.listdir(depth_folder))
original_image_size = cv2.imread(os.path.join(rgb_folder, rgb_file_list[0])).shape[:2][::-1]  # (width, height)

# resize the camera intrinsics to match the input image size
camera_intrinsics[0, 0] *= input_image_size[0] / original_image_size[0]
camera_intrinsics[1, 1] *= input_image_size[1] / original_image_size[1]
camera_intrinsics[0, 2] *= input_image_size[0] / original_image_size[0]
camera_intrinsics[1, 2] *= input_image_size[1] / original_image_size[1]

wire_detector = wdu.WireDetector(detection_config, camera_intrinsics)

wire_detection_dir = folder + 'ransac_results/'

if os.path.exists(wire_detection_dir) == False:
    os.makedirs(wire_detection_dir)

In [3]:
avg_frequency = 0.0
renderer, material = vu.create_renderer()

for file in rgb_file_list:
    start_time = time.perf_counter()
    rgb_timestamp = int(file.split('.')[0])
    closest_depth_timestamp = None
    for depth_file in depth_file_list:
        if closest_depth_timestamp is None or abs(int(depth_file.split('.')[0]) - rgb_timestamp) < abs(int(closest_depth_timestamp.split('.')[0]) - rgb_timestamp):
            closest_depth_timestamp = depth_file
            closest_depth_file = os.path.join(depth_folder, str(closest_depth_timestamp))
            
    rgb_img = cv2.imread(os.path.join(rgb_folder, file))
    depth_img = np.load(closest_depth_file)
    rgb_img = cv2.resize(rgb_img, (input_image_size[0], input_image_size[1]))
    depth_img = cv2.resize(depth_img, (input_image_size[0], input_image_size[1]))
    
    min_depth = 0.5
    depth_img[depth_img <= min_depth] = 0

    start_time = time.perf_counter()
    wire_lines, wire_midpoints, avg_angle, midpoint_dists_wrt_center = wire_detector.detect_wires_2d(rgb_img)

    regions_of_interest, roi_line_counts = wire_detector.find_regions_of_interest(depth_img, avg_angle, midpoint_dists_wrt_center)

    fitted_lines, roi_pcs, roi_point_colors, masked_rgb_img = wire_detector.ransac_on_rois(regions_of_interest, roi_line_counts, avg_angle, depth_img, viz_img=rgb_img)

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    avg_frequency += 1.0 / elapsed_time

    for fitted_line in fitted_lines:
        masked_rgb_img = vu.draw_3d_line_on_image(masked_rgb_img, fitted_line, camera_intrinsics, color=(0, 255, 0), thickness=2)

    if fitted_lines is None or len(fitted_lines) == 0 or roi_pcs is None or len(roi_pcs) == 0:
        print("No wires detected in image: ", rgb_timestamp)
        continue

    img_name_3d = os.path.join(wire_detection_dir, str(rgb_timestamp) + '_3d.png')
    vu.capture_fitted_lines_in_image(renderer, material, img_name_3d, fitted_lines, roi_pcs, roi_point_colors)
    img_name_2d = os.path.join(wire_detection_dir, str(rgb_timestamp) + '_2d.png')
    masked_rgb_img = cv2.resize(masked_rgb_img, (1920, 1080))  # Resize to 1280x720 for better visualization
    cv2.imwrite(img_name_2d, masked_rgb_img)

avg_frequency /= len(rgb_file_list)
print("Average frequency: ", avg_frequency)
print("Average Period: ", 1.0 / avg_frequency)


libEGL warning: egl: failed to create dri2 screen


[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x795d55043010 (threading is enabled)
EGL(1.5)
OpenGL(4.5)
Average frequency:  7.394374290890987
Average Period:  0.1352379472096624
